In [9]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [10]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, BitsAndBytesConfig
# from datasets import load_dataset
# from peft import UILinLoRAConfig, get_peft_model, PeftModel
# import numpy as np
# import torch
# import evaluate

In [11]:
# torch.set_printoptions(threshold=torch.inf)  # Display all elements
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.device_count()

In [12]:
# BASE_ID = "roberta-base"
# tok  = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)

# quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True)
# # quant_config = BitsAndBytesConfig(load_in_8bit=True)
# base_model = AutoModelForSequenceClassification.from_pretrained(
#     BASE_ID,
#     num_labels=2,
#     # quantization_config=quant_config,
#     device_map="auto")

In [13]:
# uilinlora_cfg = UILinLoRAConfig(
#         target_modules=["query", "value"],
#         uilinlora_alpha=1.0,
#         uilinlora_dropout=0.0,
#         fan_in_fan_out=False,
#         rank=128)
# model = get_peft_model(base_model, uilinlora_cfg)

In [14]:
# model.classifier.requires_grad_(True)

In [15]:
# for name, param in model.named_parameters():
#     if "classifier" in name:
#         print(name)
#         print(param.shape)
#         param.requires_grad = True

In [16]:
# model.train()
# batch = tok(["hello"], return_tensors="pt").to(0)
# out = model(**batch, labels=torch.tensor([1]).to(0))

# print("loss requires grad?", out.loss.requires_grad)
# print("loss grad_fn?", out.loss.grad_fn)

In [17]:
# stop

In [18]:
# for name, module in model.named_modules():
#     if hasattr(module, "uilinlora_sigma"):
#         print(f"\nAdapter module: {name}")
#         for adapter_name in module.uilinlora_sigma.keys():
#             print(f"  Adapter name: {adapter_name}")
#             print(f"    Σ shape: {module.uilinlora_sigma[adapter_name].shape}")
#             print(f"    D shape: {module.uilinlora_D[adapter_name].shape}")
#             print(f"    E shape: {module.uilinlora_E[adapter_name].shape}")
#             print(f"    U shape: {getattr(module, f'{adapter_name}_U').shape}")
#             print(f"    V shape: {getattr(module, f'{adapter_name}_V').shape}")


In [19]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"{name}: {param.shape}")

In [20]:
# for name, buf in model.named_buffers():
#     print(f"{name}: {buf.shape}")


In [21]:
# def print_trainable_params(model):
#     total = 0
#     trainable = 0
#     for name, param in model.named_parameters():
#         if "classifier" in name:
#             continue
#         total += param.numel()
#         if param.requires_grad:
#             trainable += param.numel()
#     print(f"Trainable parameters (excluding classifier): {trainable:,}")
#     print(f"Total parameters (excluding classifier): {total:,}")
#     print(f"Trainable %: {100 * trainable / total:.8f}%")


In [22]:
# print_trainable_params(model)

In [23]:
# stop

In [24]:
# model.config.pad_token_id = tok.pad_token_id

# # ---------- data ----------
# raw_ds = load_dataset("glue", "cola")

# def tok_f(ex):
#     return tok(
#         ex["sentence"],
#         truncation=True,
#         padding="max_length",
#         max_length=128
#     )

# cola_ds = load_dataset("glue", "cola")
# cola_ds = cola_ds.map(tok_f, batched=True)
# cola_ds = cola_ds.rename_column("label", "labels")
# cola_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [25]:



# # def tokenize(batch):
# #     natural   = tok(batch["sentence"], add_special_tokens=True)
# #     true_lens = [len(ids) for ids in natural["input_ids"]]

# #     padded = tok(
# #         batch["sentence"],
# #         truncation=True,
# #         padding="max_length",
# #         max_length=128,
# #     )

# #     padded["real_length"] = true_lens
# #     return padded

# # tokenized_ds = raw_ds.map(
# #     tokenize,
# #     batched=True,
# #     remove_columns=["sentence", "idx"]
# # )

# # # rename + set Torch format
# # tokenized_ds = tokenized_ds.rename_column("label", "labels")
# # tokenized_ds.set_format(
# #     type="torch",
# #     columns=["input_ids", "attention_mask", "labels", "real_length"],
# # )

# # # ---------- stats ----------
# # max_len = max(tokenized_ds["train"]["real_length"])
# # print(f"Longest raw sentence: {max_len} tokens")


# # # ---------- data ----------
# # raw_datasets = load_dataset("glue", "sst2")
# # def tokenize_function(example):
# #     return tok(example["sentence"], truncation=True, padding="max_length", max_length=100)

# # # Tokenize the entire dataset
# # tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# # del raw_datasets
# # tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# # tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [26]:
# # ---------- trainer ----------
# args = TrainingArguments(
#         output_dir="uilinlora-cola",
#         per_device_train_batch_size=32,
#         num_train_epochs=1,
#         learning_rate=3e-3,
#         eval_strategy="epoch",
#         save_strategy="no",
#         logging_steps=50)


# trainer = Trainer(model=model,
#                   args=args,
#                   train_dataset=tokenized_datasets["train"],
#                   eval_dataset=tokenized_datasets["validation"])

In [27]:
# trainer.train()

In [28]:
# stop

In [29]:
# predictions = trainer.predict(tokenized_datasets["validation"])
# logits = predictions.predictions[1]
# preds = np.argmax(logits, axis=-1)
# metric = evaluate.load("accuracy")

# tokenized_datasets["validation"]["labels"]
# metric.compute(predictions=preds, references=tokenized_datasets["validation"]["labels"])

In [30]:
# 

In [31]:
# # after trainer.train()
# adapter_dir = "uilinlora_adapter"
# model.save_pretrained(adapter_dir, safe_serialization=True)  # adapter only
# tok.save_pretrained(adapter_dir)                             # optional, for easy reload

In [32]:
# 

In [33]:
# 

In [34]:
# BASE_ID = "roberta-base"
# base = AutoModelForSequenceClassification.from_pretrained(
#            BASE_ID, num_labels=2, device_map="auto")

# model = PeftModel.from_pretrained(base, "uilinlora_adapter").to(device)
# tokenizer = AutoTokenizer.from_pretrained("uilinlora_adapter", use_fast=True)

In [35]:
# predictions = trainer.predict(tokenized_datasets["validation"])
# logits = predictions.predictions[1]
# preds = np.argmax(logits, axis=-1)
# metric = evaluate.load("accuracy")

# tokenized_datasets["validation"]["labels"]
# metric.compute(predictions=preds, references=tokenized_datasets["validation"]["labels"])

In [36]:
# 

In [37]:
# 

In [38]:
# 

In [39]:
# 

In [40]:
# # # Debugging things

# # core_model   = model.get_base_model()        # → LlamaForSequenceClassification
# # llama_blocks = core_model.model.layers       # → ModuleList of decoder layers
# # qproj_0      = llama_blocks[0].self_attn.q_proj

# # print(type(qproj_0))          # should be your Linear4bit / Linear8bitLt
# # print(qproj_0.weight.shape)   # should be (out, in)  e.g.  (4096, 2048)

# model.train()
# with torch.amp.autocast("cuda"):
#     batch = tok(["hello"], return_tensors="pt").to(0)
#     out = model(**batch, labels=torch.tensor([1]).to(0))

# loss = out.loss.to(torch.float32)
# loss.backward()


# # Check grads manually
# for name, param in model.named_parameters():
#     if param.requires_grad and param.grad is not None:
#         print(f"{name} has non-zero grad: {param.grad.abs().mean().item():.6f}")

# model.train()
# batch = tok(["hello"], return_tensors="pt").to(0)
# out = model(**batch, labels=torch.tensor([1]).to(0))

# print("loss requires grad?", out.loss.requires_grad)
# print("loss grad_fn?", out.loss.grad_fn)

# for n, p in model.named_parameters():
#     if p.requires_grad:
#         print(n, p.shape)

#         # model.train()
# # batch = tok(["hello"], return_tensors="pt").to(0)
# # out = model(**batch, labels=torch.tensor([1]).to(0))
# # print(out.loss.grad_fn)  # should NOT be None



In [41]:
# For non trained model accuracy 0.4919
# For r=128 one epoch lr 3e-3 accuracy 0.932
# For r=128 two epochs lr 3e-3 accuracy 0.939

In [42]:
# 

In [43]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch, evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, BitsAndBytesConfig, Trainer
)
from peft import UILinLoRAConfig, get_peft_model, LoraConfig, TaskType

In [44]:
torch.set_printoptions(threshold=float("inf"))
matthews = evaluate.load("matthews_correlation")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    result = matthews.compute(predictions=preds, references=labels)
    print("MCC result:", result)
    return result

# accuracy = evaluate.load("accuracy")

In [45]:
# ---------------------------  custom trainer  --------------------------- #
class UILinLoRATrainer(Trainer):
    def __init__(self, *args, head_lr=1e-3, adapter_lr=4e-3, **kw):
        super().__init__(*args, **kw)
        self.head_lr, self.adapter_lr = head_lr, adapter_lr

    def create_optimizer(self):                       # two learning rates
        if self.optimizer is None:
            head, adapter = [], []
            for n, p in self.model.named_parameters():
                if p.requires_grad:
                    (head if "classifier" in n else adapter).append(p)
            groups = [{"params": head,    "lr": self.head_lr},
                      {"params": adapter, "lr": self.adapter_lr}]
            self.optimizer = torch.optim.AdamW(groups)
        return self.optimizer

# ---------------------------  helpers  --------------------------- #
# def prepare_sst2_dataset(tokenizer, max_len=128):
#     ds = load_dataset("glue", "sst2")
#     ds = ds.map(
#         lambda ex: tokenizer(ex["sentence"],
#                              truncation=True,
#                              padding="max_length",
#                              max_length=max_len),
#         batched=True)
#     ds = ds.rename_column("label", "labels")
#     ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
#     return ds

def prepare_cola_dataset(tokenizer, max_len=128):
    ds = load_dataset("glue", "cola")
    ds = ds.map(
        lambda ex: tokenizer(
            ex["sentence"],
            truncation=True,
            padding="max_length",
            max_length=max_len,
        ),
        batched=True,
    )
    ds = ds.rename_column("label", "labels")
    ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
    return ds

In [ ]:
base_id = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(base_id, use_fast=True)

base = AutoModelForSequenceClassification.from_pretrained(
    base_id, num_labels=2, device_map="auto"
)

uilinlora_cfg = UILinLoRAConfig(
        target_modules=["query", "value"],
        rank=128,
        uilinlora_alpha=1.0,
        uilinlora_dropout=0.0,
        fan_in_fan_out=False,
        init_uilinlora_weights=True,
        task_type=TaskType.SEQ_CLS)

model = get_peft_model(base, uilinlora_cfg)
model.classifier.requires_grad_(True)   # make head trainable
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
data = prepare_cola_dataset(tokenizer)

In [ ]:
train_args = TrainingArguments(
    output_dir="uilinlora-roberta-base-cola",
    per_device_train_batch_size=64,
    num_train_epochs=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_matthews_correlation",
    greater_is_better=True,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    logging_steps=50,
    save_total_limit=1,
    seed=42,
)

trainer = UILinLoRATrainer(
    model=model,
    args=train_args,
    # train_dataset=data["train"].select(range(1000)),
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    head_lr=1e-3,
    adapter_lr=4e-3,
)

trainer.train()
# print("Best-epoch accuracy:",
#         trainer.evaluate()["eval_accuracy"])

In [ ]:
stop

In [ ]:
# data["validation"]

In [ ]:
# print(trainer.evaluate().keys())


In [ ]:
# metrics = trainer.evaluate()
# print(metrics)


In [ ]:
# output = trainer.predict(data["validation"])
# print("Predictions:", output)
